In [101]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [102]:
WANDB_PROJECT = "myprojectname"
WANDB_ENTITY = "myname"
DATASET = "fashion_mnist"
EPOCHS = 10
BATCH_SIZE = 128
LOSS = "cross_entropy"
OPTIMIZER = "sgd"
LEARNING_RATE = 0.01
MOMENTUM = 0.5
BETA = 0.5
BETA1 = 0.5
BETA2 = 0.5
EPSILON = 1e-6
WEIGHT_DECAY = 0.0
WEIGHT_INIT = "random"
NUM_LAYERS = 4
HIDDEN_SIZE = 128
ACTIVATION = "sigmoid"

In [103]:
from keras.datasets import fashion_mnist

# Load the data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

In [104]:
class FFNeuralNetwork():
    def __init__(self, 
                neurons=HIDDEN_SIZE, 
                hid_layers=NUM_LAYERS, 
                input_size=784, 
                output_size=10, 
                act_func=ACTIVATION, 
                weight_init=WEIGHT_INIT, 
                out_act_func="softmax",
                init_toggle=True):
                
        self.neurons, self.hidden_layers = neurons, hid_layers
        self.weights, self.biases = [], []
        self.input_size, self.output_size = input_size, output_size
        self.activation_function, self.weight_init = act_func, weight_init
        self.output_activation_function = out_act_func

        if init_toggle:
            self.initialize_weights()
            self.initiate_biases()

    def initialize_weights(self):
        self.weights.append(np.random.randn(self.input_size, self.neurons))
        for _ in range(self.hidden_layers - 1):
            self.weights.append(np.random.randn(self.neurons, self.neurons))
        self.weights.append(np.random.randn(self.neurons, self.output_size))

        if self.weight_init == "xavier":
            for i in range(len(self.weights)):
                self.weights[i] = self.weights[i] / np.sqrt(self.weights[i].shape[0])

    def initiate_biases(self):
        for _ in range(self.hidden_layers):
            self.biases.append(np.random.randn(self.neurons))
        self.biases.append(np.random.randn(self.output_size))
    
    def activation(self, x):
        # x is a matrix of size (batch_size, neurons)
        if self.activation_function == "sigmoid":
            return 1 / (1 + np.exp(-x))
        elif self.activation_function == "tanh":
            return np.tanh(x)
        elif self.activation_function == "ReLU":
            return np.maximum(0, x)
        else:
            raise Exception("Invalid activation function")
    
    def output_activation(self, x):
        # x is a matrix of size (batch_size, output_size)
        if self.output_activation_function == "softmax":
            max_x = np.max(x, axis=1)
            max_x = max_x.reshape(max_x.shape[0], 1)
            exp_x = np.exp(x - max_x)
            return exp_x / np.sum(exp_x, axis=1).reshape(exp_x.shape[0], 1)
        else:
            raise Exception("Invalid output activation function")
    
    def forward(self, x):
        # x is a matrix of size (batch_size, input_size)
        self.pre_activation, self.post_activation = [x], [x]

        for i in range(self.hidden_layers):
            self.pre_activation.append(np.matmul(self.post_activation[-1], self.weights[i]) + self.biases[i])
            self.post_activation.append(self.activation(self.pre_activation[-1]))
            
        self.pre_activation.append(np.matmul(self.post_activation[-1], self.weights[-1]) + self.biases[-1])
        self.post_activation.append(self.output_activation(self.pre_activation[-1]))

        return self.post_activation[-1]

In [105]:
def loss(loss, y, y_pred):
    # y is a matrix of size (batch_size, output_size)
    # y_pred is a matrix of size (batch_size, output_size)
    if loss == "cross_entropy":
        return -np.sum(y * np.log(y_pred))
    elif loss == "mean_squared":
        return np.sum((y - y_pred) ** 2) / (2 * y.shape[0])
    else:
        raise Exception("Invalid loss function")

In [106]:
class Backpropagation():
    def __init__(self, 
                 nn: FFNeuralNetwork, 
                 loss=LOSS, 
                 act_func=ACTIVATION):
        
        self.nn, self.loss, self.activation_function = nn, loss, act_func
    
    def loss_derivative(self, y, y_pred):
        # y is a matrix of size (batch_size, output_size)
        if self.loss == "cross_entropy":
            return -y / y_pred
        elif self.loss == "mse":
            return 2 * (y_pred - y)
        else:
            raise Exception("Invalid loss function")
        
    def activation_derivative(self, x):
        # x is a matrix of size (batch_size, neurons)
        if self.activation_function == "sigmoid":
            return np.exp(-x) / (1 + np.exp(-x))**2
        elif self.activation_function == "tanh":
            return 1 - np.tanh(x)**2
        elif self.activation_function == "ReLU":
            return (x > 0).astype(int)
        else:
            raise Exception("Invalid activation function")
        
    def output_activation_function(self, y, y_pred):
        # this is the derivative of the loss function with respect to the pre-activation of the output layer
        if self.nn.output_activation_function == "softmax":
            return y_pred - y
        else:
            raise Exception("Invalid output activation function")
    
    def backward(self, y, y_pred):
        # y is a matrix of size (batch_size, output_size)
        # y_pred is a matrix of size (batch_size, output_size)
        self.d_weights, self.d_biases = [], []
        self.d_h, self.d_a = [], []

        self.d_h.append(self.loss_derivative(y, y_pred))
        self.d_a.append(self.output_activation_function(y, y_pred))

        for i in range(self.nn.hidden_layers, 0, -1):
            self.d_weights.append(np.matmul(self.nn.post_activation[i].T, self.d_a[-1]))
            self.d_biases.append(np.sum(self.d_a[-1], axis=0))
            self.d_h.append(np.matmul(self.d_a[-1], self.nn.weights[i].T))
            self.d_a.append(self.d_h[-1] * self.activation_derivative(self.nn.pre_activation[i]))

        self.d_weights.append(np.matmul(self.nn.post_activation[0].T, self.d_a[-1]))
        self.d_biases.append(np.sum(self.d_a[-1], axis=0))

        self.d_weights.reverse()
        self.d_biases.reverse()

        return self.d_weights, self.d_biases

In [107]:
class Optimiser():
    def __init__(self, 
                 nn: FFNeuralNetwork, 
                 bp:Backpropagation, 
                 lr=LEARNING_RATE, 
                 optimiser=OPTIMIZER, 
                 momentum=MOMENTUM):
        
        self.nn, self.lr, self.optimiser = nn, lr, optimiser
        self.momentum = momentum
        self.bp = bp
        self.history_weights = [np.zeros_like(w) for w in self.nn.weights]
        self.history_biases = [np.zeros_like(b) for b in self.nn.biases]

    def run(self, d_weights, d_biases, y, x):
        if(self.optimiser == "sgd"):
            self.SGD(d_weights, d_biases)
        elif(self.optimiser == "momentum"):
            self.MomentumGD(d_weights, d_biases)
        elif(self.optimiser == "nag"):
            self.NAG(y, x)
        else:
            raise Exception("Invalid optimiser")
    
    def SGD(self, d_weights, d_biases):
        for i in range(self.nn.hidden_layers + 1):
            self.nn.weights[i] -= self.lr * d_weights[i]
            self.nn.biases[i] -= self.lr * d_biases[i]

    def MomentumGD(self, d_weights, d_biases):
        for i in range(self.nn.hidden_layers + 1):
            self.history_weights[i] = self.momentum * self.history_weights[i] + d_weights[i]
            self.history_biases[i] = self.momentum * self.history_biases[i] + d_biases[i]

            self.nn.weights[i] -= self.history_weights[i] * self.lr
            self.nn.biases[i] -= self.history_biases[i] * self.lr

    def NAG(self, y, x):
        nn_new = FFNeuralNetwork(neurons=self.nn.neurons,
                                 input_size=self.nn.input_size,
                                 output_size=self.nn.output_size,
                                 hid_layers=self.nn.hidden_layers,
                                 act_func=self.nn.activation_function,
                                 out_act_func=self.nn.output_activation_function,
                                 weight_init=self.nn.weight_init,
                                 init_toggle=False)
        
        bp_new = Backpropagation(nn=nn_new, 
                                 loss=self.bp.loss,
                                 act_func=self.bp.activation_function)
        
        nn_new.weights = [w - self.momentum * self.history_weights[i] for i, w in enumerate(self.nn.weights)]
        nn_new.biases = [b - self.momentum * self.history_biases[i] for i, b in enumerate(self.nn.biases)]

        y_pred_new = nn_new.forward(x)
        d_weights_new, d_biases_new = bp_new.backward(y, y_pred_new)

        for i in range(self.nn.hidden_layers + 1):
            self.history_weights[i] = self.momentum * self.history_weights[i] + d_weights_new[i]
            self.history_biases[i] = self.momentum * self.history_biases[i] + d_biases_new[i]

            self.nn.weights[i] -= self.history_weights[i] * self.lr
            self.nn.biases[i] -= self.history_biases[i] * self.lr     

In [108]:
def train(x_train, y_train, parameters):
    # x_train is a matrix of size (batch_size, input_size)
    # y_train is a matrix of size (batch_size, output_size) - one-hot encoded

    nn = FFNeuralNetwork(input_size=parameters['input_size'], 
                         hid_layers=parameters['hidden_layers'], 
                         neurons=parameters['neurons'], 
                         output_size=parameters['output_size'], 
                         act_func=parameters['activation'], 
                         out_act_func=parameters['output_activation'])
    bp = Backpropagation(nn=nn, 
                         loss=parameters['loss'],
                         act_func=parameters['activation'])
    opt = Optimiser(nn=nn,
                    bp=bp,
                    lr=parameters['learning_rate'],
                    optimiser=parameters['optimiser'],
                    momentum=parameters['momentum'])
    
    batch_size = parameters['batch_size']

    print("Initial Accuracy: {}".format(np.sum(np.argmax(nn.forward(x_train), axis=1) == np.argmax(y_train, axis=1)) / y_train.shape[0]))

    for epoch in range(10):
        for i in range(0, x_train.shape[0], batch_size):
            x_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]

            y_pred = nn.forward(x_batch)
            d_weights, d_biases = bp.backward(y_batch, y_pred)
            opt.run(d_weights, d_biases, y_batch, x_batch)

        y_pred = nn.forward(x_train)
        print("Epoch: {}, Loss: {}".format(epoch + 1, loss("cross_entropy", y_train, y_pred)))
        print("Accuracy: {}".format(np.sum(np.argmax(y_pred, axis=1) == np.argmax(y_train, axis=1)) / y_train.shape[0]))
    
    return nn

In [109]:
x_train_reshape = x_train.reshape(x_train.shape[0], -1)

y_train_reshape = np.zeros((y_train.shape[0], 10))
y_train_reshape[np.arange(y_train.shape[0]), y_train] = 1 # one-hot encoding

parameters = {
    'input_size': 784,
    'hidden_layers': 1,
    'neurons': 4,
    'output_size': 10,
    'activation': 'sigmoid',
    'output_activation': 'softmax',
    'loss': 'cross_entropy',
    'learning_rate': 0.01,
    'optimiser': 'nag',
    'momentum': 0.2,
    'batch_size': 128
}

nn = train(x_train_reshape[:128], y_train_reshape[:128], parameters)

Initial Accuracy: 0.09375
Epoch: 1, Loss: 360.5018494573545
Accuracy: 0.1328125
Epoch: 2, Loss: 369.4307913758029
Accuracy: 0.0859375
Epoch: 3, Loss: 364.1020186490009
Accuracy: 0.0703125
Epoch: 4, Loss: 368.2301158424348
Accuracy: 0.0703125
Epoch: 5, Loss: 347.71273635030957
Accuracy: 0.0390625
Epoch: 6, Loss: 348.93141286645937
Accuracy: 0.0234375
Epoch: 7, Loss: 339.2993457788411
Accuracy: 0.0546875
Epoch: 8, Loss: 335.56200820887153
Accuracy: 0.109375
Epoch: 9, Loss: 329.38092626993136
Accuracy: 0.1015625
Epoch: 10, Loss: 336.37813776604986
Accuracy: 0.1328125


In [82]:
x_test_reshape = x_test.reshape(x_test.shape[0], -1)

y_pred = nn.forward(x_test_reshape)
print("Test Accuracy: {}".format(np.sum(np.argmax(y_pred, axis=1) == y_test) / y_test.shape[0]))

Test Accuracy: 0.1
